Ejemplo 1: Validacion

In [2]:
import functools

# Aspecto que valida que los números sean positivos
def validate_positive_numbers(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        # Validar argumentos posicionales
        for arg in args:
            if isinstance(arg, (int, float)) and arg < 0:
                raise ValueError(f"El número {arg} no es positivo.")
        # Validar argumentos de palabra clave
        for key, value in kwargs.items():
            if isinstance(value, (int, float)) and value < 0:
                raise ValueError(f"El valor {value} de \"{key}\" no es positivo.")
        return func(*args, **kwargs)
    return wrapper


In [3]:
# Clase Calculadora de Areas
class calculadora_areas:
    @validate_positive_numbers
    def area_circulo(self, radio):
        return 2 * 3.14159 * radio**2

    @validate_positive_numbers
    def area_rectangulo(self, base, altura):
        return base * altura

    @validate_positive_numbers
    def area_trapecio(self, base_menor, base_mayor, altura):
        return altura * (base_mayor + base_menor) / 2

# Utilizar la clase calculadora_areas
calc = calculadora_areas()
print(calc.area_circulo(10))
print(calc.area_rectangulo(10, 20))
print(calc.area_trapecio(10, 20, 30))


628.318
200
450.0


Ejemplo 2: Seguridad

In [2]:
import functools

# Diccionario de usuarios válidos
users = {'admin': '12345'}

# Decorador de autenticación
def authenticate():
    def decorator(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            username = input("Ingrese usuario: ")
            password = input("Ingrese contraseña: ")
            if username in users and users[username] == password:
                return func(*args, **kwargs)
            else:
                raise PermissionError("Usuario o contraseña inválidos")
        return wrapper
    return decorator


In [3]:
@authenticate()
class Configuracion:
    def __init__(self):
        self.igv = 0.18
        self.uit = 5150

    def __str__(self):
        return f"IGV : {self.igv*100} % \nUIT : S/. {self.uit}"

    def modificar_igv(self, igv):
        self.igv = igv

    def modificar_uit(self, uit):
        self.uit = uit

conf = Configuracion()
conf.modificar_igv(0.17)
conf.modificar_uit(5000)
print(conf)


Ejemplo 3: Registro

In [9]:
import functools
import datetime

def registro(func):
    @functools.wraps(func)
    def wrapper_registro(*args, **kwargs):
        resultado = func(*args, **kwargs)
        accion = func.__name__
        time = datetime.datetime.now()
        if accion == 'agregar_nota':
            nota = args[1]
            print(f"[LOG: {time}] Se agrego la nota: {nota}")
        elif accion == 'eliminar_nota':
            nota = args[1]
            if resultado:
                print(f"[LOG: {time}] Se elimino la nota: {nota}")
            else:
                print(f"[LOG: {time}] Se intento eliminar la nota: {nota}")
        return resultado
    return wrapper_registro


In [10]:
class registro_notas:
    def __init__(self, docente=''):
        self.docente = docente
        self.notas = []

    @registro
    def agregar_nota(self, nota):
        self.notas.append(nota)

    @registro
    def eliminar_nota(self, nota):
        if nota in self.notas:
            self.notas.remove(nota)
            return True
        else:
            return False

    def __str__(self):
        return str(self.notas)

notas_matematicas = registro_notas()
notas_matematicas.agregar_nota(5)
notas_matematicas.agregar_nota(12)
notas_matematicas.agregar_nota(14)
notas_matematicas.agregar_nota(16)
notas_matematicas.eliminar_nota(5)
notas_matematicas.eliminar_nota(11)
print(notas_matematicas)


[LOG: 2024-06-12 00:26:08.043677] Se agrego la nota: 5
[LOG: 2024-06-12 00:26:08.043677] Se agrego la nota: 12
[LOG: 2024-06-12 00:26:08.043677] Se agrego la nota: 14
[LOG: 2024-06-12 00:26:08.043677] Se agrego la nota: 16
[LOG: 2024-06-12 00:26:08.043677] Se elimino la nota: 5
[LOG: 2024-06-12 00:26:08.043677] Se intento eliminar la nota: 11
[12, 14, 16]


Ejemplo 4: Transacciones

In [12]:
import functools

# Decorador de transaccion
def transactional(func):
    @functools.wraps(func)
    def wrapper_transactional(*args, **kwargs):
        # Iniciar transaccion
        print("\nIniciando transaccion")
        try:
            # Ejecutar la funci ́on decorada
            result = func(*args, **kwargs)
            cantidad = args[1]
            accion = func.__name__
            mi_almacen.transacciones.append((accion, cantidad))
            # Finalizar transacci on con exito
            print("Transaccion exitosa")
            return result
        except Exception as e:
            # Manejar excepci ́on y revertir transacci ́on
            print(f"Error en transaccion: {e}")
            print("Revirtiendo transaccion")
            mi_almacen.revertir_operaciones()
            raise
        finally:
            # Finalizar transacci ́on
            print("Finalizando transaccion")
            return wrapper_transactional


In [13]:
class Almacen:
    def __init__(self, stock=0):
        self.stock = stock
        self.transacciones = [] # Lista para registrar transacciones

    @transactional
    def ingreso(self, cantidad):
        if cantidad <= 0:
            raise ValueError("La cantidad de productos debe ser positiva")
        self.stock += cantidad
        print(f"Ingreso: {cantidad}. Nuevo stock: {self.stock}")

    @transactional
    def salida(self, cantidad):
        if cantidad > self.stock:
            raise ValueError("Stock insuficiente")
        self.stock -= cantidad
        print(f"Salida: {cantidad}. Nuevo stock: {self.stock}")

    def revertir_operaciones(self):
        # Revertir todas las transacciones registradas
        while self.transacciones:
            tipo, cantidad = self.transacciones.pop()
            if tipo == 'ingreso':
                self.stock -= cantidad
            elif tipo == 'salida':
                self.stock += cantidad


In [14]:
mi_almacen = Almacen(100)
print(f"\nStock inicial: {mi_almacen.stock}")


Stock inicial: 100


In [15]:
try:
    mi_almacen.ingreso(50)
    mi_almacen.salida(30)
    mi_almacen.salida(150)
except Exception as e:
    print(f"Operacion fallida: {e}")
print(f"\nStock final: {mi_almacen.stock}")

Operacion fallida: 'NoneType' object is not callable

Stock final: 100
